In [0]:
from google.colab import drive
drive.mount('drive/')

In [0]:
import os
import pandas as pd
import numpy as np

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")

In [0]:
votes.head(2)

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D,iou,best
0,1164,6132,62,656,276,803,47,650,213,812,0.988892,225,36,3969,81,1,1,0,1,214.0,147.0,722.0,31458.0,1.455782,259.624729,0.613974,1
1,1399,6132,184,662,235,740,47,650,213,812,0.936663,18769,144,484,5184,0,0,1,0,51.0,78.0,258.0,3978.0,0.653846,93.193347,0.079069,0


In [0]:
item_count = votes.itemId.value_counts()
item_count.describe()

count    941.000000
mean       5.653560
std        2.678333
min        2.000000
25%        3.000000
50%        5.000000
75%        8.000000
max       24.000000
Name: itemId, dtype: float64

In [0]:
item_iou = votes.groupby('itemId')['iou'].mean()
item_iou.describe()

count    941.000000
mean       0.477921
std        0.125622
min        0.085098
25%        0.395072
50%        0.474036
75%        0.555142
max        0.872342
Name: iou, dtype: float64

In [0]:
item_count.corr(item_iou) 

-0.3969776601430035

In [0]:
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)
votest.head(2)

,userId,itemId,Xmin,Ymin,Xmax,Ymax,L,H,P,S,L/H,D
0,1581,34804,86,640,242,743,156,103,518,16068,1.514563,186.935818
1,1351,34804,85,655,273,766,188,111,598,20868,1.693694,218.323155


In [0]:
votest.itemId.value_counts().describe()

count    630.000000
mean       5.719048
std        2.547255
min        1.000000
25%        3.000000
50%        6.000000
75%        8.000000
max       14.000000
Name: itemId, dtype: float64

In [0]:
votest.shape # (3603, 12)

(3603, 12)

In [0]:
#Нормализация данных
def norm(train_data, test_data):
    mean = train_data.mean(axis=0)
    train_data -= mean
    std = train_data.std(axis=0)
    train_data /= std
    test_data -= mean
    test_data /= std
    return train_data, test_data

In [0]:
data1 = votes[['Xmin', 'L',	'H', 'P', 'S', 'L/H', 'D']]
test1 = votest[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']]

data2 = votes[['Ymin', 'L',	'H', 'P', 'S', 'L/H', 'D']]
test2 = votest[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']]

data3 = votes[['Xmax', 'L',	'H', 'P', 'S', 'L/H', 'D']]
test3 = votest[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']]

data4 = votes[['Ymax', 'L',	'H', 'P', 'S', 'L/H', 'D']]
test4 = votest[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']]

In [0]:
data1 = norm(data1, test1)[0]
test1 = norm(data1, test1)[1]

data2 = norm(data2, test2)[0]
test2 = norm(data2, test2)[1]

data3 = norm(data3, test3)[0]
test3 = norm(data3, test3)[1]

data4 = norm(data4, test4)[0]
test4 = norm(data4, test4)[1]

In [0]:
dlist = [data1, data2, data3, data4]
tlist = [test1, test2, test3, test4]
ylist = [ydata1, ydata2, ydata3, ydata4]

In [0]:
ydata1 = votes.Xmin_true
ydata2 = votes.Ymin_true
ydata3 = votes.Xmax_true
ydata4 = votes.Ymax_true

In [0]:
from keras import models
from keras import layers

Using TensorFlow backend.


In [0]:
def build_model():
 model = models.Sequential()
 model.add(layers.Dense(64, activation='relu',
 input_shape=(X_train.shape[1],)))
 model.add(layers.Dense(64, activation='relu'))
 model.add(layers.Dense(1))
 model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
 return model

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
def split(X, Y):
    return train_test_split(X, Y, test_size=1717, random_state=42) # X_train, X_test, y_train, y_test

In [0]:
MODELS = []
for i in range(4):
    X_train, X_test, y_train, y_test = split(dlist[i], ylist[i])
    model = build_model()
    model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1)
    val_mse, val_mae = model.evaluate(X_test, y_test, verbose=1)
    print(val_mse, val_mae)
    MODELS.append(model)

In [0]:
Xpmin = MODELS[0].predict(data1)
Ypmin = MODELS[1].predict(data2)
Xpmax = MODELS[2].predict(data3)
Ypmax = MODELS[3].predict(data4)

In [0]:
answers = votes[['itemId','Xmin','Ymin','Xmax','Ymax','Xmin_true','Ymin_true','Xmax_true','Ymax_true']]	

In [0]:
answers['Xpmin'] =  Xpmin
answers['Ypmin'] =  Ypmin
answers['Xpmax'] =  Xpmax
answers['Ypmax'] =  Ypmax

In [0]:
answers["iou"] = answers[['Xmin','Ymin','Xmax','Ymax','Xmin_true','Ymin_true','Xmax_true','Ymax_true']]	.apply(intersection_over_union, axis=1)
answers["iou"].mean()

## СО СВЕРТОЧНЫМИ СЛОЯМИ


*  Токены = объекты (itemId)
*  Они должны быть одинаковой длины, поэтому где мало сэмплов - дополнить 0



In [0]:
from keras.layers import Embedding
embedding_layer = Embedding(1000, 64)  (пакет с 1000 последовательностями, каждая длиной 64)  or  (32, 10) (пакет с 32 , каждая длиной 10) или (64, 15)

Все последовательности в пакете должны иметь одинаковую длину, потому что упаковываются в один
тензор, поэтому короткие последовательности, если они есть, нужно дополнить
нулями, а длинные — усечь. В частности, они варьируются от 2 до 24...

In [0]:
tokens = len(votes.itemId.unique()) + len(votest.itemId.unique())
tokens # 1571, можно задать 1600?

1571

In [0]:
max_features = 2000 # в предположении, что "словарь" состоит из координат от 0 до 2000
maxlen = votes.itemId.value_counts().max()
maxlen = 8 # 24 

2

In [0]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
model = Sequential()
# Определение максимальной длины входа для слоя Embedding в целях последующего уменьшения размерности. После слоя Embedding: активация имеет форму (образцы, максимальная_длина, 8)
model.add(Embedding(max_features, 2, input_length=maxlen)) # 2 - двухмерное векторное представление
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
metrics=['acc'])
model.summary()
history = model.fit(x_train, y_train,
 epochs=10,
 batch_size=32,
 validation_split=0.2)

